# Word Representations

## Statistical Language Models:

#### Counting Frequencies:


<table>
    <tr>
        <td>            
            <img src="images/document-classification-matrix.png">
        </td>
        <td>
            <img src="images/there-was-a-crooked-man.png">
        </td>
    <tr>
</table>

The above table can be turned into a matrix where rows correspond to a single word and columns correspond to a document. The values in the cell represent the frequency of a word in a particular document.

This tends to work well for document classification problems.

### N-Gram Model &mdash; Counting Consecutive Word Pair Frequencies:


<table>
    <tr>
        <td width="50%">  
            <p style="text-align: center;">
                <strong>
                    Consecutive Word Pair Frequency Matrix
                </strong>
            </p>          
            <img src="images/consecutive-word-frequency.png">
            <p style="text-align: left;">
                This matrix counts the occurrences of one word appear immediately after another. Eg. "a" is followed immediately by "crooked" 6 times in the input text.
            </p>
        </td>
        <td>
            <p style="text-align: center;">
                <strong>
                    Bi-gram Model
                </strong>
            </p>
            <img src="images/1-gram-consecutive-word-frequency.png">
            <p style="text-align: left;">
                The bigram model takes the *consecutive word pair frequency matrix* and divides every entry by the sum of the row's values. This produces a probability distribution for each row.
            </p>
        </td>
    </tr>
</table>


With the bi-gram model, we can estimate the probability of any word $w_j$ appearing after word $w_i$. For example, "a" has a $\frac{6}{7}\approx 85.7\%$ chance of being followed by "crooked". 

__N-gram__ &mdash; a contiguous sequence of N words. The N-gram model has $N$ dimensions and is used to store the probability of different sequences of $N$ consecutive words occurring together in a document.
- N-gram models are language models that can represent the context of words to a language processing network
    - Eg. in the string "we need to book our tickets soon" and "we need to read this book soon", the word "book" has different semantics depending on the surrounding words
    - Bi-gram models look at every contiguous pair of words, while tri-gram models look at every contiguous triplet of words. In general, the larger the value of $N$, the more unique the set of words are, meaning you'll tend to fail to recognise important *pairs* of words.  <img src="images/unigram-bigram-trigram.png" width="50%">

- It's necessary to aggregate statistics across a large collection of similar documents to prevent unusual words like "crooked" from dominating the probability distribution
- A bigram model using the English vocabulary would typically be of size $60000 \times 60000$
- An N-gram model would be of size $\underbrace{60000 \times 60000 \times \ldots \times 60000}_{N times}$, taking exponentially more memory as $N$ increases



### Co-occurrence Matrix



<table>
    <tr>
        <td>            
            <p style="text-align: center;">2-Word-Window Co-Occurrence Matrix</p>
            <img src="images/co-occurrence-matrix-2-window.png">
        </td>
        <td>
            <p style="text-align: center;">10-Word-Window Co-Occurrence Matrix</p>
            <img src="images/co-occurrence-matrix-10-window.png">
        </td>
    <tr>
</table>

A *co-occurrence matrix* stores the probability of a word occurring near another word instead of focusing specifically on whether $w_j$ appears after $w_i$ like for the N-gram model. In other words, with a *co-occurrence matrix*, we only care about the set of words appearing around a particular central word, rather than the particular order that words occur in. 

A window size of $2$ would count 1 word on either side of a central word. A window size of $10$ would count 5 words on either side of the central word.

Each word's corresponding row in the co-occurrence matrix can be regarded as the vector representation of that word. With the co-occurrence matrix, words that appear in similar contexts tend to have similar vector representations.

The word vector dimensions can be on the order of $10^5$ or larger, depending on the size of the vocabulary. One purpose of *word embeddings* is to find vector representations for words which don't have such a ridiculously large dimension.

## Word Embeddings:

The goal of *word embeddings* is to find a vector representation for each word where 'nearby' representations are likely to occur in similar contexts. *Word embeddings* are *vector representations* for words.

A vector representation for "apple" might look like $[0.7, 1.2, -0.345, ...]$, for example. This word embedding may consist of any number of elements, this is a parameter that we decide. An interesting consequence of numericalising words with a technique like *Word2Vec* is that the Euclidean distance between any 2 word vectors represents similarity. So vector operations can give you results like $\texttt{apple + purple} \approx \texttt{plum}$.

### Singular Value Decomposition:
Singular value decomposition is frequently used for *dimensionality reduction*.

In linear algebra, the *singular value decomposition* is a factorisation of any matrix $X=USV^T$ of dimensions $L \times M$ where $U_{L\times L}$ and $V_{M \times M}$ are square *unitary* matrices and $S_{L \times M}$ is a *diagonal* matrix with non-negative entries $s_1, s_2, ..., s_M \geq 0$.
- A square matrix $A$ is unitary if the inverse is equal to the conjugate transpose: $A^{-1}=(\overline {A})^T$
    - The sum of the squares of the values in each row is equal to 1
    - All rows are orthogonal to each other
- In the case of a co-occurrence matrix $X$, the dimensions are $L\times L$, where $L$ is the size of the lexicon


The goal is to obtain an __approximation__ for matrix $X$ of a much __smaller rank__ $N<M$ by:
- Truncating $U_{L \times L}$ to $\widetilde{U}_{L \times N}$ a matrix with far fewer columns
- Truncating $S_{L \times M}$ to $\widetilde{S}_{N \times N}$ a matrix with far fewer columns and rows
- Truncating $V^T_{M \times M}$ to $\widetilde{V}^T_{N \times M}$, a matrix with far fewer rows

The product $\widetilde{U}_{L \times N} \cdot \widetilde{S}_{N \times N} \cdot \widetilde{V}^T_{N \times M}$ is a matrix of size $L \times M$, the same dimensions as the original matrix $X$. 

The $k^{th}$ row of $\widetilde{U}$ gives the vector representing word $k$ of the lexicon.

<img src="images/singlar-value-decomposition.png" width="75%">





<table>
    <tr>
        <td width="50%">
            <p style="text-align: center;">10-Word-Window Co-Occurrence Matrix</p>
            <img src="images/co-occurrence-matrix-10-window.png">
        </td>
        <td>            
            <p style="text-align: center;">Word vectors of size 2 obtained after running SVD on the co-occurrence matrix</p>
            <img src="images/singlar-value-decomposition-example.png" width="100%">
        </td>
    <tr>
</table>


The time complexity of SVD is $O(L\times M^2)$, which is impractical if a large lexicon size like $L=60000$ is used. Two alternative ways to generate word vectors are *Word2Vec* and *GloVe*.

### Word2Vec:




<img src="images/word2vec-1-word-context-model.png" width="50%">


Given a word, this network predicts what other words in the lexicon are likely to appear next to that word.

- The inputs are 1-hot encoded.
- There are no activations. This hidden layer values are a purely linear function of the input layer, and the output layer values are a purely linear function of the hidden layer.

- The $k^{th}$ word is represented by the $k^{th}$ row in the matrix $W_{V\times N}$, which we'll call $v_k$. Note that $V$ is the size of the lexicon and $N$ is the number of hidden nodes (the dimension we want to compress to).

- The $j^{th}$ column of matrix $W'_{N\times V}$ is an *alternative* vector representation of the $j^{th}$ word, which we'll call $v'_j$.

- The output probability for a particular word $y_j$ being nearby word $k$ is given by: $y_j=v'^T_j v_k$.


#### Word2Vec replacing SVD:

With Word2Vec, the input-to-hidden weight matrix takes the place of $\widetilde{U}$ and the hidden-to-output weights takes the place of $\widetilde{V}^T$ in SVD. 

Instead of carrying out regular SVD, we're trying to *learn the values* of $\widetilde{U}$ and $\widetilde{V}$ with a neural network.

#### Cost Function:

Softmax can be used to normalise the output layer values into a probability distribution. Given the input word, the Word2Vec network outputs the probabilty of word $y_j$ occurring next to the input word as: 

$$prob(y_j) = \frac{e^{u_j}}{\sum_{i=i}^V e^{u_i}}.$$


Suppose the dictionary is numbered from 0-60000 and each word in any document can be mapped to a specific number. Each document can then be considered as just a sequence of numbers: $w_1, w_2, ..., w_T$ where $w_i=j, j\in [0, 60000]$ with $j$ being the $j^{th}$ word in the dictionary.

We then train the network to learn weights that would maximise: 

$$
    \frac{1}{T}\sum_{t=1}^T \big(\sum_{-c\leq r \leq c, r \neq 0} \log \text{prob}(w_{t+r}|w_t)\big)
$$
where $c$ is the number of words we want to look at before and after the middle word and $T$ is the number of words in the document. Put simply, we're training the network to output high probabilities for words $w_{t+r}$ within the context window that occur around an input word $w_t$.


Working with vanilla softmax, if the true output value is $j^*$, then the cost function is:

$$
    E = -u_{j^*} + \log{\sum_{i=1}^V e^{u_{i}}}.
$$

- The gradient for each output $u_j$ is given by 
$$e_j=\frac{\partial E}{\partial u_j}=\delta_{jj^*} + \frac{\partial}{\partial u_j} \log{\sum_{i=1}^V e^{u_i}},$$
where $\delta_{jj^*} = 1$ if the prediction equals the true value $j=j^*$, otherwise $\delta_{jj^*}=0$.

- The gradient for hidden-to-output weights:
$$
    \frac{\partial E}{\partial w'_{ij}} = \frac{\partial E}{\partial u_j} \cdot \frac{\partial u_j}{\partial w'_{ij}} = e_j h_i
$$

- The gradient for hidden unit activations:
$$
    \frac{\partial E}{\partial h_i} = \sum_{j=1}^V \big( \frac{\partial E}{\partial u_j} \cdot \frac{\partial u_j}{\partial h_i} \big) = \sum_{j=1}^V \big( e_j w'_{ij} \big)
$$

- The gradient for input-to-hidden weights:
$$
    \frac{\partial E}{\partial w_{ki}} = \frac{\partial E}{\partial h_i} \cdot \frac{\partial h_i}{\partial w_{ki}} = \sum_{j=1}^V \big( e_j w'_{ij x_k} \big)
$$

At the completion of training, we typically only keep the input-to-hidden weights and discard the hidden-output weights. The input-to-hidden weights already gives us the word embeddings we're after.

Note: having to compute softmax on an output layer of size 60000 for every word in the text is computationally expensive. Alternatives to softmax are:
- Hierarchical softmax
- Negative sampling

Both hierarchical softmax and negative sampling are techniques for significantly reducing the amount of computation necessary for training the Word2Vec network, by orders of magnitudes.

The time complexity for vanilla softmax on the output layer is $O(V)$, where $V$ is the size of the vocabulary. With hierarchical softmax, the time complexity is $O(\log_2 V)$ 

#### Hierarchical Softmax:

Hierarchical softmax is applicable to any network that uses softmax at the output layer. It's just a technique for approximating the softmax function.



<table>
    <tr>
        <td width="50%">        
            <p style="text-align: center;">Softmax visualised as a single layer tree</p>
            <img src="images/softmax-tree.png" width="100%">
        </td>
        <td>
            <p style="text-align: center;">Hierarchical softmax visualised as a Huffman coding tree</p>
            <img src="images/hierarchical-softmax-tree.png" width="100%">
        </td>
    <tr>
</table>


All the words in the vocabulary are organised into a Huffman tree, based on the frequency each word in the input text body.

<img src="images/hierarchical-softmax-huffman-tree.png" width="50%">


We start at the root node. At each internal node, shaded grey, we compute the probability that the correct word is in left subtree or the right subtree by passing the output node's weighted sum through the sigmoid function. The sigmoid output tells you the probability of that the correct word is in the right subtree.

With this, we're now training the Word2Vec model to produce an output layer that can produce the correct sigmoid activations that can guide you down the Huffman tree to the correct word. 

The product of the sigmoid activations in the path down to a leaf is the probability of that word occurring in the context of the input word.

<img src="images/hierarchical-softmax-huffman-tree-probability-path-example.png" width="50%">
<em><p style="text-align: center;">Here, the network thinks that word $w_2$ has a $0.56 \times 0.23 \times 0.68 \approx 8.75\%$ chance of occurring based on whatever word it received as input</p></em>


Remember, the goal of Word2Vec is not to output a probability distribution for all words, we're just trying to create good word vectors.

#### Negative Sampling:

A hacky alternative to hierarchical that also produces good word embeddings with significantly less compute time.

Instead of passing through $60000$ inputs, we might sample $20$ words at random that didn't occur with the input word &mdash; this is called *negative sampling*.

The goal now is to train the network such that the output of the correct class is larger and the output for the incorrect class is lower, using the following error function:

$$
    E = \underbrace{-\log \big( \sigma(v'^T_{j^*} \cdot h) \big)}_{\text{Positive samples correctly predicted}} - \underbrace{\sum_{j\in W_{\text{neg}}} \log \big( \sigma(-v'^T_j \cdot h)\big),}_{\text{Negative samples correctly predicted}} 
$$

where $W_{\text{neg}}$ is a set of negative samples.

#### Continuous Bag of Words and Skip-Gram:

The *continuous bag of words* model and *skip-gram* model are 2 different extensions of Word2Vec. Note that Word2Vec only predicts 1 word. These 2 extensions predict multiple words, given 1 input word. 

<table>
    <tr>
        <td>            
            <p style="text-align: center;"><strong>Continuous Bag of Words</strong></p>
            <img src="images/continuous-bag-of-words.png">
        </td>
        <td>
            <p style="text-align: center;"><strong>Skip-Gram</strong></p>
            <img src="images/skip-gram.png" width="100%">
        </td>
    <tr>
</table>


Suppose we have a context window of size 10.
- *Continuous bag of words* &mdash; given 10 surrounding words as input, we're predicting the word in the middle of those 10 context words
- *Skip-gram* &mdash; given the centre word as input, we're predicting the 10 surrounding words. This is the opposite task of *continuous bag of words*
- Continuous bag of words and skip gram produce more or less the same results. Which one is used depends on which one is faster in the application.


### Resources:

- <a href="https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469">Understanding word vectors</a>
- <a href="https://towardsdatascience.com/hierarchical-softmax-and-negative-sampling-short-notes-worth-telling-2672010dbe08">Hierarchical softmax and negative sampling</a>

